In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   preco            10000 non-null  float64
 1   vendido          10000 non-null  int64  
 2   idade_do_modelo  10000 non-null  int64  
 3   km_por_ano       10000 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 312.6 KB


In [4]:
dados.describe()

,preco,vendido,idade_do_modelo,km_por_ano
count,10000.000000,10000.000000,10000.00000,10000.000000
mean,64842.373698,0.580000,13.85580,22825.898794
std,25250.592393,0.493583,4.68758,8060.494332
min,-840.360000,0.000000,1.00000,584.190420
25%,44796.375000,0.000000,11.00000,16857.434165
50%,65225.685000,1.000000,15.00000,21594.928790
75%,85111.237500,1.000000,18.00000,27643.230845
max,118929.720000,1.000000,20.00000,63684.802480


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
x = dados.drop(columns="vendido",axis=1)
y = dados[["vendido"]]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=101)
#divisao simples dos dados de treino e test,tal divisão é muito suscetível à aleatoriedade

In [11]:
#treinando um arvore para servir de exemplo
from sklearn.tree import DecisionTreeClassifier
SEED = 158020
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(x_train,y_train)
previsoes = modelo.predict(x_test)
print(accuracy_score(y_test,previsoes))
#o uso de diferentes valores para a seed implica em diferentes valores da acuracia
#do modelo,o que indica a influencia da aleatoriedade

0.7636


In [18]:
#usando o cross_validate para diminuir a influencia da aleatoriedade
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo,x,y,cv=10)
resultados
#os resultados da acuracia para cada  um dos testes podem ser vistos em "test_score"

{'fit_time': array([0.02907586, 0.01404905, 0.01370788, 0.0118444 , 0.01252556,
        0.01117134, 0.01496458, 0.01087928, 0.01068377, 0.01090002]),
 'score_time': array([0.00479865, 0.00274396, 0.00297022, 0.00667238, 0.00243711,
        0.00225163, 0.00276065, 0.00221133, 0.00206971, 0.00218582]),
 'test_score': array([0.742, 0.77 , 0.749, 0.764, 0.761, 0.764, 0.754, 0.755, 0.759,
        0.76 ])}

In [19]:
#atraves da coluna test_score podemos extrair o "intervalo de confiança
#basta fazer a media+- duas vezes o desvio padrao
media = resultados["test_score"].mean()
desvio_padrao = resultados["test_score"].std()
desvio_padrao
extremo_inferior = media - 2*desvio_padrao
extremo_superior = media + 2*desvio_padrao
intervalo = [extremo_inferior,extremo_superior]
print(intervalo)

[0.7424168923815764, 0.7731831076184235]


In [21]:
#utilizando o KFold para inserir a aleatoriedade no cross_validate
#o KFold possuir um parametro que insere a aleatoriedade(embaralha os dados)
from sklearn.model_selection import KFold
cv = KFold(n_splits=10,shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo,x,y,cv=cv)
media = resultados["test_score"].mean()
desvio_padrao = resultados["test_score"].std()
desvio_padrao
extremo_inferior = media - 2*desvio_padrao
extremo_superior = media + 2*desvio_padrao
intervalo = [extremo_inferior,extremo_superior]
print(intervalo)

[0.7339035567500098, 0.7816964432499902]


In [23]:
#utilizando a estratificação para corrigir o problema de desbalanceamento dos dados
#podemos usar o parametro "stratified" do train_test_split,ou usar o stratified_kfold
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10,shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo,x,y,cv=cv)
media = resultados["test_score"].mean()
desvio_padrao = resultados["test_score"].std()
extremo_inferior = media - 2*desvio_padrao
extremo_superior = media + 2*desvio_padrao
intervalo = [extremo_inferior,extremo_superior]
print(intervalo)

[0.7278827808778957, 0.7877172191221044]


In [25]:
#criando uma nova coluna com dados "aleatorios" para o modelo do carro
#a criaçao dessa coluna permite uma maior separação entre os dados
dados["modelo"] = dados["idade_do_modelo"] + np.random.randint(-2,3,size=10000)
dados["modelo"].unique()

array([17, 21, 10,  5, 16, 12, 14,  1, 19,  8,  3,  7, 20, 18, 15,  9, 13,
       11,  4,  6, 22,  2,  0, -1])

In [27]:
#somaremos 1 em cada dado para trabalharmos somente com valores não negativos
dados["modelo"] = dados["modelo"] + abs(dados["modelo"].min()) + 1
dados["modelo"].unique()

array([19, 23, 12,  7, 18, 14, 16,  3, 21, 10,  5,  9, 22, 20, 17, 11, 15,
       13,  6,  8, 24,  4,  2,  1])

In [32]:
y = dados["vendido"]
x = dados.drop(columns=["vendido","modelo"],axis=1)
x.head(2)

,preco,idade_do_modelo,km_por_ano
0,30941.02,18,35085.22134
1,40557.96,20,12622.05362


In [ ]:
#quando utilizamos o kfold e o stratifiedkfold,os dados de treino e de teste
#possuem todos os grupos de carro,o que não é bom,precisamos analisar o desempenho
#do modelo quando este for confrontado com modelos não vistos ainda
#para solucionar esse problema utilizaremos o groupkfold

In [39]:
from sklearn.model_selection import GroupKFold
cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo,x,y,cv=cv,groups=dados["modelo"])
media = resultados["test_score"].mean()
desvio_padrao = resultados["test_score"].std()
desvio_padrao
extremo_inferior = media - 2*desvio_padrao
extremo_superior = media + 2*desvio_padrao
intervalo = [extremo_inferior,extremo_superior]
print(intervalo)
print(media)

[0.7157676757459434, 0.800029614260877]
0.7578986450034102
